# Exercise 3 Submitted for Pranjal Sharma, Alper Daggez, Deniz Gaiser and Anthony Akpenyi

**Please Note**: We updated the requirements.txt

Please install the new requirements before editing this exercise.

## Import packages

In [2]:
import os

from vll.utils.download import download_mnist
import numpy as np
import matplotlib.pyplot as plt

import skimage
import skimage.io

import torch
import torch.nn.functional as F
from torchvision import transforms

from models.mnist.simple_cnn import Net

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Task 1
(2 points)

In this task, you will learn some basic tensor operations using the PyTorch library.

Reference for torch: https://pytorch.org/docs/stable/torch.html

In [3]:
# Create a numpy array that looks like this: [0, 1, 2, ..., 19]
arr = np.arange(0,20)
print(arr)
# Convert the numpy array to a torch tensor
tensor = torch.tensor(arr)
print(tensor)

# Create a tensor that contains random numbers.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor.
rand_tensor = torch.rand(20)
tensor = torch.mul(tensor,rand_tensor)
print(tensor)

# Create a tensor that contains only 1s.
# It should have the same size like the numpy array.
# Substract it from the previous tensor.
tensor = tensor - torch.full((20,),1)
print(tensor)

# Get the 5th element using a index.
element = tensor[4]
print(element)

# Create a tensor that contains only 0s.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor without any assignment (in place).
print(torch.mul(tensor, torch.full((20,),0)))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
tensor([ 0.0000,  0.6264,  0.5604,  1.4975,  3.2536,  2.2088,  0.9175,  1.4606,
         3.3471,  6.1724,  1.8064,  7.0809,  7.8575,  7.9777,  5.4407,  2.8247,
        11.3769, 10.6537, 17.6102, 15.7071])
tensor([-1.0000, -0.3736, -0.4396,  0.4975,  2.2536,  1.2088, -0.0825,  0.4606,
         2.3471,  5.1724,  0.8064,  6.0809,  6.8575,  6.9777,  4.4407,  1.8247,
        10.3769,  9.6537, 16.6102, 14.7071])
tensor(2.2536)
tensor([-0., -0., -0., 0., 0., 0., -0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [21]:
# Load the image from the last exercise as RGB image.
image = skimage.io.imread("data/pepo.jpg")

# Convert the image to a tensor
image = torch.tensor(image)
# Print its shape

print(image.shape)
original_shape = image.shape

# Flatten the image
image = image.view(-1, 512*512*3)
print(image.shape)
print(len(image))
print(image)
# Add another dimension resulting in a 1x78643 tensor

image = image[None,:,None]
image = torch.reshape(image, (1, 786432))
print(image.shape)

# Revert the last action
image.view(-1)
print(image.shape)

# Reshape the tensor, so that it has the original 2D dimensions
image = image.view(original_shape)
print(image.shape)

# Calculate the sum, mean and max of the tensor
print(torch.sum(image))
print(image.float().mean())
print(image.float().max())

torch.Size([512, 512, 3])
torch.Size([1, 786432])
1
tensor([[183, 194, 118,  ..., 221, 237, 162]], dtype=torch.uint8)
torch.Size([1, 786432])
torch.Size([1, 786432])
torch.Size([512, 512, 3])
tensor(91404741)
tensor(116.2271)
tensor(255.)


## Task 2
(2 points)

Use Autograd to perform operations on a tensor and output then gradients.

In [92]:
# Create a random 2x2 tensor which requires gradients
x = torch.rand(2,2, requires_grad=True)

print(x)

# Create another tensor by adding 2.0
y = x + 2.0
print(y)

# Create a third tensor z = y^2
z = y ** 2
print(z)

# Compute out as the mean of values in z
out = torch.mean(z)
print(out)

# Perform back propagation on out
out.backward()

# Print the gradients dout/dx
gradient = x.grad
print(gradient)

# Create a copy of y whithout gradients
y2 = y.detach()
print(y2.requires_grad)

# Perform the mean operation on z

print(torch.mean(z))
# with gradients globally disabled
with torch.no_grad():
    print(torch.mean(z))

tensor([[0.0451, 0.1978],
        [0.3840, 0.0834]], requires_grad=True)
tensor([[2.0451, 2.1978],
        [2.3840, 2.0834]], grad_fn=<AddBackward0>)
tensor([[4.1823, 4.8303],
        [5.6833, 4.3408]], grad_fn=<PowBackward0>)
tensor(4.7592, grad_fn=<MeanBackward0>)
tensor([[1.0225, 1.0989],
        [1.1920, 1.0417]])
False
tensor(4.7592, grad_fn=<MeanBackward0>)
tensor(4.7592)


## Task 3
(3 points)

Implement a Dataset class for MNIST.

In [52]:
# We first download the MNIST dataset
download_mnist()

Extract mnist.tar.gz
Done!


In [96]:
class MNIST:
    """
    Dataset class for MNIST
    """

    def __init__(self, root, transform=None):
        """
        root -- path to either "training" or "testing"
        
        transform -- transform (from torchvision.transforms)
                     to be applied to the data
        """
        # save transforms
        self.transform = transform
        
        # TODO: create a list of all subdirectories (named like the classes) 
        #       within the dataset root
        dataset_root = 'data'
        subdirectories = [d for d in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, d))]

        # 'subdirectories' now contains a list of subdirectory names within the 'dataset_root'
                                                                                                                                                
        print(subdirectories)
        
        # TODO: create a list of paths to all images
        #       with the ground truth label
        image_paths_with_labels = []
        for root, dirs, files in os.walk(dataset_root):
            for file in files:
                if file.endswith(".jpg"): 
                    image_path = os.path.join(root, file)
                    label = os.path.basename(root)
                    image_paths_with_labels.append((image_path, label))

    
    def __len__(self):
        """
        Returns the lenght of the dataset (number of images)
        """
        # TODO: return the length (number of images) of the dataset
        return image_path_with_labels.shape

    def __getitem__(self, index):
        """
        Loads and returns one image as floating point numpy array
        
        index -- image index in [0, self.__len__() - 1]
        """
        # TODO: load the ith image as an numpy array (dtype=float32)
        ith_image_path, ith_image_label = image_paths_with_labels[index]
        ith_image = Image.open(ith_image_path)
        ith_image_array = np.array(ith_image)
        # TODO: apply transforms to the image (if there are any)
        transform = transforms.Compose([ transforms.Resize((224, 224)), transforms.ToTensor(), ])
        transformed_image = transform(ith_image)
        # TODO: return a tuple (transformed image, ground truth)
        my_tuple = [transformed_image,ith_image_label ]
        return my_tuple

## Task 4
(3 points)

You can now load a pretrained neural network model we provide.
Your last task is to run the model on the MNIST test dataset, plot some example images with the predicted labels and compute the prediction accuracy.

In [98]:
def validate(model, data_loader):
    # TODO: Create a 10x10 grid of subplots
   
    fig, axes = plt.subplots(10, 10, figsize=(10, 10))

    for i in range(10):
        for j in range(10):
            axes[i, j].text(0.5, 0.5, f'Subplot ({i+1}, {j+1})', ha='center', va='center')


    model.eval()
    correct = 0 
    
    with torch.no_grad():
        for i, item in enumerate(data_loader):
            # TODO: unpack item into image and ground truth
            #       and run network on them
            
            
            # TODO: get class with highest probability
            
            
            # TODO: check if prediction is correct
            #       and add it to correct count
            
            
            # plot the first 100 images
            if i < 100:
                # TODO: compute position of ith image in the grid
                pos = __getitem__(1)
                
                # TODO: convert image tensor to numpy array
                #       and normalize to [0, 1]
                
                
                # TODO: make wrongly predicted images red
                
                
                # TODO: disable axis and show image
                
                
                # TODO: show the predicted class next to each image
                
            
            elif i == 100:
                plt.show()
    
    # TODO: compute and print the prediction accuracy in percent
    

# create a DataLoader using the implemented MNIST dataset class
data_loader = torch.utils.data.DataLoader(
    MNIST('data/mnist/testing',
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=1, shuffle=True)

# create the neural network
model = Net()

# load the statedict from 'models/mnist/simple_cnn.pt'
model.load_state_dict(torch.load('models/mnist/simple_cnn.pt'))

# validate the model
validate(model, data_loader)

IndentationError: expected an indented block (1458944782.py, line 45)